# Ryan plays around with MLFlow
SageMaker has been dumped... or has it? Anyway let's get the data out of lakeFS.

In [39]:
import pandas as pd
import lakefs_client
from lakefs_client.model.commit_creation import CommitCreation
import mlflow
from lakefs_client.client import LakeFSClient
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pickle
from io import BytesIO  
import mlflow
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = ''
configuration.password = ''
configuration.host = '' # yeah so you do need to make a hole in the security group
                                                 # even if the two ec2 instances are in the same SG.
                                                 # These are private IPs, if you are wondering - hopefully this
                                                 # does not go over the net...
client = LakeFSClient(configuration)

In [3]:
# This grabs the data.csv file out of the main branch of the countries repository in lakeFS 
file = client.objects.get_object('countries','main','data.csv')
df = pd.read_csv(file)
df

,CID,Longitude,Latitude
0,500,-30.044039,24.216976
1,500,150.388214,-21.728706
2,500,168.733444,56.352604
3,500,-34.896107,4.820187
4,500,3.984333,-26.328730
...,...,...,...
260994,91,-27.684013,83.460632
260995,91,-29.777115,83.461273
260996,91,-33.087837,83.464043
260997,91,-33.384373,83.475655


In [4]:
df = df.replace(500, 252)
df = df.sample(frac=1)
df

,CID,Longitude,Latitude
42062,190,-5.752711,-15.979079
115106,12,-69.920410,12.480947
159954,137,0.683665,21.115690
38165,81,-149.356155,-17.699827
179444,111,34.979435,30.486445
...,...,...,...
76476,204,103.758972,1.409751
7755,252,168.207245,83.513908
115911,12,-70.035515,12.568818
123794,192,-61.018837,13.857307


In [5]:
features = df[["Longitude", "Latitude"]].to_numpy()
target = df[["CID"]].to_numpy()

In [6]:
clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))

In [7]:
clf.fit(features, target.ravel())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier())])

In [8]:
print(clf.predict([[-98, 36]]))

[65]


Hmm interesting. What is 65 anyway?

In [12]:
# This grabs the countries.csv file out of the main branch of the countries repository in lakeFS 
file = client.objects.get_object('countries','main','countries.csv')
countries = pd.read_csv(file)
countries.loc[countries['CID'] == 65]

,CID,COUNTRY
64,65,Dominica


Hahaha. That is smack dab in the middle of the USA. This is going to be great.

I am going to update the countries csv with the new key for ocean, and then commit these changes for both the data and the keys to lakeFS.

In [15]:
# switching from the main branch to the working branch
client.objects.upload_object('countries','working','data.csv', 
                             content=BytesIO(df.to_csv(index=False).encode('utf-8')))

{'checksum': 'c755cb02819e1689e308d355e3dfb29e',
 'content_type': 'application/octet-stream',
 'mtime': 1701541555,
 'path': 'data.csv',
 'path_type': 'object',
 'physical_address': 's3://ryans-lakefs-test/countries/data/gjhoojif4ngu7qoas2mg/cllndcif4ngu7qoas2n0',
 'size_bytes': 6754629}

In [16]:
client.objects.upload_object('countries','working','countries.csv', 
                             content=BytesIO(countries.to_csv(index=False).encode('utf-8')))

{'checksum': '9d8ac4c86f669711a1fd35633d5c6f97',
 'content_type': 'application/octet-stream',
 'mtime': 1701541572,
 'path': 'countries.csv',
 'path_type': 'object',
 'physical_address': 's3://ryans-lakefs-test/countries/data/gjhoojif4ngu7qoas2mg/cllndh2f4ngu7qoas2ng',
 'size_bytes': 3609}

In [20]:
client.commits_api.commit('countries', 'working', 
                         CommitCreation(message='Updated data.csv and countries.csv')
                         )

{'committer': 'test-user',
 'creation_date': 1701542034,
 'id': 'de845f0d13f88f38636bd12cfb72cb46c6ab0ebbea356bccd4c8281bbefdc3d8',
 'message': 'Updated data.csv and countries.csv',
 'meta_range_id': '',
 'metadata': {},
 'parents': ['200a6679e879047bdd2225ac91a951ab39bd74356dc8471233f5d4ab99573a46']}

In [21]:
client.refs_api.diff_refs(repository='countries', left_ref='main', right_ref='working')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 1},
 'results': [{'path': 'data.csv',
              'path_type': 'object',
              'size_bytes': 6754629,
              'type': 'changed'}]}

In [22]:
countries = countries.replace(500, 252)
countries

,CID,COUNTRY
0,1,Afghanistan
1,2,Albania
2,3,Algeria
3,4,American Samoa
4,5,Andorra
...,...,...
247,248,Wallis and Futuna
248,249,Yemen
249,250,Zambia
250,251,Zimbabwe


In [24]:
client.objects.upload_object('countries','working','countries.csv', 
                             content=BytesIO(countries.to_csv(index=False).encode('utf-8')))

{'checksum': 'd4efb1160fd95e2ddf3019e066d11a39',
 'content_type': 'application/octet-stream',
 'mtime': 1701542610,
 'path': 'countries.csv',
 'path_type': 'object',
 'physical_address': 's3://ryans-lakefs-test/countries/data/gjhoojif4ngu7qoas2mg/cllnlkif4ngu7qoas2qg',
 'size_bytes': 3609}

Oookay. Time time mess around with MLFlow. This model is absolutely terrible, and I am loving it.

Okay let's do things their way. I think real men don't test and just deploy, but whatever. https://mlflow.org/docs/latest/getting-started/intro-quickstart/notebooks/tracking_quickstart.html

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, target.ravel(), test_size=0.2, random_state=42)

In [36]:
params = {} # no hyperparameters for this guy

In [37]:
y_pred = clf.predict(X_test)

In [42]:
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.03885057471264368

Absolutely awful. Love it!!!

In [43]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://172.31.92.133:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Countries")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Predict countries")

    # Infer the model signature
    signature = infer_signature(X_train, clf.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="countries_model",
        signature=signature,
        input_example=features,
        registered_model_name="tracking-countries",
    )

Successfully registered model 'tracking-countries'.
2023/12/02 19:37:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-countries, version 1
Created version '1' of model 'tracking-countries'.


Well... that went smoother than I thought. Not that I really care, but let's have a closer look.

In [44]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [46]:
predictions = loaded_model.predict(X_test)

feature_names = ['lon', 'lat']

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:10]

,lon,lat,actual_class,predicted_class
0,177.754059,-17.431265,77,77
1,159.882172,-9.700796,208,191
2,-78.123383,24.624023,17,65
3,27.555864,61.573971,78,124
4,126.664238,38.245316,165,124
5,-73.444840,-46.934589,47,75
6,-8.826954,7.314955,127,41
7,-62.989243,17.494589,189,65
8,69.982071,30.833944,170,124
9,-13.343016,10.354385,97,41


Well this thing needs to be right at least some of the time for it to be fun... this just looks like complete garbage but we'll deploy it anyway then start tweaking. 